In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [11]:
from FilesToUploadToColab.experiments import DirectTemplate, EnumeratedTemplate, Experiment
from pytorch_pretrained_bert import BertForMaskedLM, GPT2LMHeadModel
import pandas as pd

In [3]:
bert = BertForMaskedLM.from_pretrained('bert-large-uncased')
gpt = GPT2LMHeadModel.from_pretrained('gpt2')

In [8]:
data_dir = "../FilesToUploadToColab/dev_sample.txt"
relation_dir = "../FilesToUploadToColab/relation_map_multiple.json"
device = 'cpu'

sentences_tail = EnumeratedTemplate(
    data_dir, 
    False, 
    True, 
    device, 
    model = gpt, 
    template_loc=relation_dir
)
sentences_head = EnumeratedTemplate(
    data_dir, 
    True, 
    False, 
    device, 
    model = gpt, 
    template_loc=relation_dir
)
experiment = Experiment(sentences_head, sentences_tail, bert, device)
experiment.run_experiment()

(-7.466170787811279, -3.6693410873413086, '[CLS] the tool is an object . [SEP]', 1)
-11.135511875152588
(-5.335318088531494, -1.6787428855895996, '[CLS] you are likely to find a fe ##rret at a pet store . [SEP]', 1)
-7.014060974121094
(-7.293811798095703, -2.6744000911712646, '[CLS] if you want to get onto web then you should turn on your computer . [SEP]', 1)
-9.968211889266968
(-6.5516157150268555, -0.21354389190673828, '[CLS] helium is a gas . [SEP]', 1)
-6.765159606933594
(-9.518251419067383, -4.429260730743408, '[CLS] most rock is hard . [SEP]', 1)
-13.947512149810791
(-11.424589157104492, -12.518746376037598, '[CLS] one of the things you do when you case joint is criminal . [SEP]', 0)
-23.94333553314209
(-5.562908172607422, -4.671208381652832, '[CLS] something that might happen while the lizard is sunni ##ng itself on rock . [SEP]', 0)
-10.234116554260254
(-7.574993133544922, -6.372175693511963, '[CLS] go jo ##g requires a person . [SEP]', 0)
-13.947168827056885
(-5.1884913444519

In [16]:
df = pd.DataFrame(experiment.results, columns = ('head_prob', 'tail_prob', 'sent', 'label'))
df['pmi_squared'] = df.head_prob + df.tail_prob

In [17]:
df.sort_values('pmi_squared')

,head_prob,tail_prob,sent,label,pmi_squared
11,-9.620617,-15.595428,[CLS] beer can cry loudly . [SEP],0,-25.216044
5,-12.518746,-11.424589,[CLS] one of the things you do when you case j...,0,-23.943336
9,-7.586630,-12.061570,[CLS] the carpet requires going to school . [SEP],0,-19.648201
4,-4.429261,-9.518251,[CLS] most rock is hard . [SEP],1,-13.947512
7,-6.372176,-7.574993,[CLS] go jo ##g requires a person . [SEP],0,-13.947169
0,-3.669341,-7.466171,[CLS] the tool is an object . [SEP],1,-11.135512
8,-5.389935,-5.188491,[CLS] the wall requires building . [SEP],0,-10.578427
6,-4.671208,-5.562908,[CLS] something that might happen while the li...,0,-10.234117
2,-2.674400,-7.293812,[CLS] if you want to get onto web then you sho...,1,-9.968212
1,-1.678743,-5.335318,[CLS] you are likely to find a fe ##rret at a ...,1,-7.014061


In [29]:
sent, mask, ids, label = sentences_head[0]
pred_head = bert(mask.reshape(1,-1), ids.reshape(1,-1)).log_softmax(2)

In [32]:
sent, mask, ids, label = sentences_tail[0]
pred_tail = bert(mask.reshape(1,-1), ids.reshape(1,-1)).log_softmax(2)

In [35]:
sentences_head.id_to_text(mask)

'[CLS] the tool is an [MASK] . [SEP]'

In [ ]:
pred_head[0,2,:]

In [36]:
pred_tail[0,5,:]

tensor([-14.5999, -14.8616, -15.0442,  ..., -14.7977, -15.0979, -13.2881],
       grad_fn=<SliceBackward>)

In [37]:
sentences_head.id_to_text(pred_tail[0,5,:].topk(100)[1])

'example axe acronym object engine invention amplifier algorithm instrument explosive umbrella envelope arrow encyclopedia idea integer oven anchor apparatus illusion ##vil option egg automatic eye expert apple attachment animal arm implant elevator opener iv analogy item artifact airplane accelerator antenna experiment accessory orb enigma index accordion organ automobile iphone android innovation interface imp image alternative investment ax organism aircraft argument interceptor expression array app oracle implement edge equation rpg alphabet amulet indicator exercise alias x organizer atlas injection adjective instant element ultimatum exception application orgasm inhibitor emergency icon adaptation ace asteroid actor absolute emerald explosion elastic illustration asset imperative elephant'